<a href="https://colab.research.google.com/github/KwonYN/TIL/blob/master/PYTHON/TENSORFLOW2/03_02_char_rnn_7_kyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **실습 3-2 : Char-RNN**



## **Import Module**

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

tf.__version__

TensorFlow 2.x selected.


'2.0.0'

## **DataSet**

### 학습할 문장 만들기

In [0]:
# 학습할 문장
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)
print ("Length of 'test sentence' is %s" %len(sentence))

FOLLOWING IS OUR TRAINING SEQUENCE:
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
Length of 'test sentence' is 180


### 입력 문자열과 타겟 문자 준비

#### Char dictionary 만들기

In [0]:
# make charater dictionary 
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
print ("CHARACTERS: ")
print (len(char_set))
print (char_set)
print ("DICTIONARY: ")
print (len(char_dic))
print (char_dic)

CHARACTERS: 
25
['t', 'd', 'f', 'u', "'", 'c', 'b', 'p', ',', ' ', 'r', 'e', 'i', 'k', 'h', 'w', 'y', 'g', 's', 'a', 'l', 'o', 'n', 'm', '.']
DICTIONARY: 
25
{'t': 0, 'd': 1, 'f': 2, 'u': 3, "'": 4, 'c': 5, 'b': 6, 'p': 7, ',': 8, ' ': 9, 'r': 10, 'e': 11, 'i': 12, 'k': 13, 'h': 14, 'w': 15, 'y': 16, 'g': 17, 's': 18, 'a': 19, 'l': 20, 'o': 21, 'n': 22, 'm': 23, '.': 24}


#### 문자열

In [0]:
data_dim    = len(char_set) # train data X:input
num_classes = len(char_set) # trian data Y:target
sequence_length = 10        # any arbitrary number
print ('data_dim : %d' %data_dim)
print ('num_classes : %d' %num_classes)

data_dim : 25
num_classes : 25


In [0]:
dataX = []  # input sequence list
dataY = []  # target sequence list 

# we will make 170 sequences 
for i in range(0, len(sentence) - sequence_length):
    # 10 characters = 1 training sequence
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1 : i+sequence_length+1]
    # convert x, y str to index(int)
    x_idx = [char_dic[c] for c in x_str] 
    y_idx = [char_dic[c] for c in y_str] 
    # append to dataset list
    dataX.append(x_idx)
    dataY.append(y_idx)

    # monitoring
    if i<5:
        print ("[%3d/%3d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*10, x_idx, y_idx))

[  0/180] [if you wan]=>[f you want]
          [12, 2, 9, 16, 21, 3, 9, 15, 19, 22]=>[2, 9, 16, 21, 3, 9, 15, 19, 22, 0]
[  1/180] [f you want]=>[ you want ]
          [2, 9, 16, 21, 3, 9, 15, 19, 22, 0]=>[9, 16, 21, 3, 9, 15, 19, 22, 0, 9]
[  2/180] [ you want ]=>[you want t]
          [9, 16, 21, 3, 9, 15, 19, 22, 0, 9]=>[16, 21, 3, 9, 15, 19, 22, 0, 9, 0]
[  3/180] [you want t]=>[ou want to]
          [16, 21, 3, 9, 15, 19, 22, 0, 9, 0]=>[21, 3, 9, 15, 19, 22, 0, 9, 0, 21]
[  4/180] [ou want to]=>[u want to ]
          [21, 3, 9, 15, 19, 22, 0, 9, 0, 21]=>[3, 9, 15, 19, 22, 0, 9, 0, 21, 9]


In [0]:
# check type and data
print('\n')
print ((type(dataX)))
print (dataX[0])
print (dataX[1])
print (dataX[168])
print (dataX[169])

print('\n')
print ((type(dataY)))
print (dataY[0])
print (dataY[1])
print (dataY[168])
print (dataY[169])



<class 'list'>
[12, 2, 9, 16, 21, 3, 9, 15, 19, 22]
[2, 9, 16, 21, 3, 9, 15, 19, 22, 0]
[9, 21, 2, 9, 0, 14, 11, 9, 18, 11]
[21, 2, 9, 0, 14, 11, 9, 18, 11, 19]


<class 'list'>
[2, 9, 16, 21, 3, 9, 15, 19, 22, 0]
[9, 16, 21, 3, 9, 15, 19, 22, 0, 9]
[21, 2, 9, 0, 14, 11, 9, 18, 11, 19]
[2, 9, 0, 14, 11, 9, 18, 11, 19, 24]


### 입력문자 list를 3-dim로, 타겟문자 list를 2-dim로 변환  

In [0]:
# input tensor 생성 
X = np.array(dataX)                                      # ndarray(170,10)<-[170,10]
# convert list sequence to 3dim array(one-hot coding)
sequences = [tf.keras.utils.to_categorical(
                 x, num_classes = data_dim) for x in X]  # list[170,10,25]
X = np.array(sequences)                                  # ndarray(170,10,25)
print("shape of X :", X.shape)

# Target tensor 생성
y = np.array(dataY)[:,-1]                                # ndarray(170,)<-[170,10]
print("shape of y :", y.shape)

# convert the vector to 2-dim 
y = tf.keras.utils.to_categorical(
                  y, num_classes = data_dim)             # (170,25)

print ("shape of y :", y.shape) # (170, 25)
print (y[0])    # t -> one-hot ,  (if you wan't')

shape of X : (170, 10, 25)
shape of y : (170,)
shape of y : (170, 25)
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


## **Model**

### Define

In [0]:
# Sequenctial model define: LSTM + Dense
hidden_dim = 75
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=hidden_dim,
                               input_shape=(sequence_length,num_classes)))
model.add(tf.keras.layers.Dense(data_dim, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 75)                30300     
_________________________________________________________________
dense (Dense)                (None, 25)                1900      
Total params: 32,200
Trainable params: 32,200
Non-trainable params: 0
_________________________________________________________________


### Compile

In [0]:
model.compile(loss='categorical_crossentropy', # one-hot coding
              optimizer='adam', 
              metrics=['accuracy'])

### Fit

Epoch 1000/1000   
170/170 [==============================] - 0s 235us/sample - loss: 0.0012 - accuracy: 1.0000    
CPU times: user 48.3 s, sys: 6.54 s, total: 54.8 s    
Wall time: 47.8 s (@Notebook Setting/GPU)

In [0]:
%%time
model.fit(X,            # X.shape : (170, 10, 25)
          y,            # y.shape : (170, 25)
          epochs=200)

Train on 170 samples
Epoch 1/200
170/170 [==============================] - 3s 16ms/sample - loss: 3.2127 - accuracy: 0.1059
Epoch 2/200
170/170 [==============================] - 0s 174us/sample - loss: 3.1773 - accuracy: 0.1941
Epoch 3/200
170/170 [==============================] - 0s 189us/sample - loss: 3.1407 - accuracy: 0.1882
Epoch 4/200
170/170 [==============================] - 0s 174us/sample - loss: 3.0859 - accuracy: 0.1882
Epoch 5/200
170/170 [==============================] - 0s 150us/sample - loss: 3.0002 - accuracy: 0.1882
Epoch 6/200
170/170 [==============================] - 0s 156us/sample - loss: 2.9003 - accuracy: 0.1882
Epoch 7/200
170/170 [==============================] - 0s 163us/sample - loss: 2.8553 - accuracy: 0.1882
Epoch 8/200
170/170 [==============================] - 0s 171us/sample - loss: 2.8229 - accuracy: 0.1882
Epoch 9/200
170/170 [==============================] - 0s 221us/sample - loss: 2.8046 - accuracy: 0.1882
Epoch 10/200
170/170 [=============

## **Generate Text:** Analysis

### Define generate_seq function

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, char_dic, seq_length, seed_text, n_chars):
	in_text = seed_text									# seed text + generated text
	# generate a fixed number of characters
	for i in range(n_chars):
		# encode the characters as integers
		encoded = [char_dic[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		encoded = tf.keras.utils.to_categorical(encoded, num_classes=len(char_dic))
		#encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		for char, index in char_dic.items():
			if index == yhat:
				break
		# append to input
		in_text += char
	return in_text

### Generate a sequence of characters

학습문장: if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

In [0]:
# test start of rhyme
print('Trigger characters: "want to bu "\nResult: \n"{}"'.format( 
      generate_seq(model, char_dic, 10, 'want to bu', 200)))
# test mid-line
print('\nTrigger characters: "collect wo"\nResult: \n"{}"'.format(
      generate_seq(model, char_dic, 10, 'collect wo', 100)))
# test not in original
print('\nTrigger characters: "rather than"\nResult: "{}"'.format(
      generate_seq(model, char_dic, 10, 'rather than', 200)))
# test end of rhyme
print('Trigger characters: "immensity of the sea."\nResult: \n"{}"'.format( 
      generate_seq(model, char_dic, 10, 'immensity of the sea.', 360)))

Trigger characters: "want to bu "
Result: 
"want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.assniishe mt mtmgg 'fo ottee  d  d sm"

Trigger characters: "collect wo"
Result: 
"collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of "

Trigger characters: "rather than"
Result: "rather thanh thes aaklsntir hhhra ctclllletooo rrr  ldleaceestt iht hhe so llgggheetettccs i  oooo o dooddnn'd oogspp tteeeeeees siiiitnit    uunnss  t kke,,    hhhhee  googg tteoooeec   wwoim   ddoond'  soong t"
Trigger characters: "immensity of the sea."
Result: 
"immensity of the sea.assniishe mt mtmgg 'fo ottee  d  d smme' syyy f ttttoeead    aimn,     bbb,e o hhiheetoo  ttt  ra.al leaahhhehm tao oggddrrettottttcl d dddd  aasspp,d enndnda mmmkpppeeyftttoet  ld   sssee ne iisiii tt   d hassm ''ttte  a  a ssh ee nmyy  iiheggee tttttem s

## **실습과제**

### 과제 1. 더 긴문장을 생성하도록 해보자
####-- 학습문장이 " ...  immensity of the sea."로 끝났다. 첫번째 생성문장이 뒷부분이 이상하다. 180자로 학습한 한계 때문일까? 
#### -- 학습문장 전체를 한번더 반복하여 학습문장을 360자로 확장하고 결과를 비교해 보자
#### -- 결과를 분석하자: 




In [0]:
# test end of rhyme
print('Trigger characters: "immensity of the sea."\nResult: \n"{}"'.format( 
      generate_seq(model, char_dic, 10, 'immensity of the sea.', 360)))

Trigger characters: "immensity of the sea."
Result: 
"immensity of the sea.assniishe mt mtmgg 'fo ottee  d  d smme' syyy f ttttoeead    aimn,     bbb,e o hhiheetoo  ttt  ra.al leaahhhehm tao oggddrrettottttcl d dddd  aasspp,d enndnda mmmkpppeeyftttoet  ld   sssee ne iisiii tt   d hassm ''ttte  a  a ssh ee nmyy  iiheggee tttttem sld  ssnf,, d drbbbut ttor  aollleeethh r smsslleeeyttsh mis l.sygeftettttth d ddlll aatsp hheeeeeggg tii"


### 과제 2. 세번째 생성문장은 여전히 문제가 많다. 모델을 개선해 보자 
####-- LSTM layer를 추가해 보자 
####-- 더 무엇을 해볼까? 생각해 보자
####-- 결과를 분석하자:

## **Model**

### Define

In [0]:
# Sequenctial model define: LSTM + Dense
hidden_dim = 75
model_2 = tf.keras.models.Sequential()
model_2.add(tf.keras.layers.LSTM(units=hidden_dim,
                               input_shape=(sequence_length,num_classes), return_sequences=True))
model_2.add(tf.keras.layers.LSTM(units=hidden_dim, return_sequences=True))
model_2.add(tf.keras.layers.LSTM(units=hidden_dim))
model_2.add(tf.keras.layers.Dense(data_dim, activation='softmax'))

model_2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 10, 75)            30300     
_________________________________________________________________
lstm_8 (LSTM)                (None, 10, 75)            45300     
_________________________________________________________________
lstm_9 (LSTM)                (None, 75)                45300     
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1900      
Total params: 122,800
Trainable params: 122,800
Non-trainable params: 0
_________________________________________________________________


### Compile

In [0]:
model_2.compile(loss='categorical_crossentropy', # one-hot coding
              optimizer='adam', 
              metrics=['accuracy'])

### Fit

Epoch 1000/1000   
170/170 [==============================] - 0s 235us/sample - loss: 0.0012 - accuracy: 1.0000    
CPU times: user 48.3 s, sys: 6.54 s, total: 54.8 s    
Wall time: 47.8 s (@Notebook Setting/GPU)

In [0]:
%%time
model_2.fit(X,            # X.shape : (170, 10, 25)
            y,            # y.shape : (170, 25)
            epochs=200)

Train on 170 samples
Epoch 1/200
170/170 [==============================] - 4s 22ms/sample - loss: 3.2048 - accuracy: 0.1765
Epoch 2/200
170/170 [==============================] - 0s 292us/sample - loss: 3.1398 - accuracy: 0.1882
Epoch 3/200
170/170 [==============================] - 0s 262us/sample - loss: 2.9726 - accuracy: 0.1882
Epoch 4/200
170/170 [==============================] - 0s 264us/sample - loss: 2.9003 - accuracy: 0.1882
Epoch 5/200
170/170 [==============================] - 0s 268us/sample - loss: 2.8716 - accuracy: 0.1882
Epoch 6/200
170/170 [==============================] - 0s 257us/sample - loss: 2.8492 - accuracy: 0.1882
Epoch 7/200
170/170 [==============================] - 0s 286us/sample - loss: 2.8368 - accuracy: 0.1882
Epoch 8/200
170/170 [==============================] - 0s 334us/sample - loss: 2.8326 - accuracy: 0.1882
Epoch 9/200
170/170 [==============================] - 0s 324us/sample - loss: 2.8225 - accuracy: 0.1882
Epoch 10/200
170/170 [=============

In [0]:
# test end of rhyme
print('Trigger characters: "immensity of the sea."\nResult: \n"{}"'.format( 
      generate_seq(model_2, char_dic, 10, 'immensity of the sea.', 200)))
# test start of rhyme
print('Trigger characters: "want to bu "\nResult: \n"{}"'.format( 
      generate_seq(model_2, char_dic, 10, 'want to bu', 200)))
# test mid-line
print('\nTrigger characters: "collect wo"\nResult: \n"{}"'.format(
      generate_seq(model_2, char_dic, 10, 'collect wo', 100)))
# test not in original
print('\nTrigger characters: "rather than"\nResult: "{}"'.format(
      generate_seq(model_2, char_dic, 10, 'rather than', 200)))

Trigger characters: "immensity of the sea."
Result: 
"immensity of the sea. ssii tth  tt  o o.sa aa. tdmww,b mm ouuuemffotlosst tofdtttt o  m     attat  r  lppheeeeetee. iiii  iitt   tta   arr d h eaccea ssimogttn'tnfoosmm   dgoedf  ts cntd  tat  b dr dharh eecdoccesi dfnog "
Trigger characters: "want to bu "
Result: 
"want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea. ssii tth  tt  o o.sa aa. tdmww,b mm "

Trigger characters: "collect wo"
Result: 
"collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of "

Trigger characters: "rather than"
Result: "rather thanhlteesn l stiiiehotty   ao   s ooona  't nn  hsm ut e .ghosss  cc .to''.l os    ttttt   s     a     ooooo  ttwwww ta  uuuuu   ttttttt o       taa    rr  he  eece   i ii s  rttt e neep  siiit ee tt   m"


In [0]:
# Data를 늘려도, Layer를 더 늘려도 처음과 결과가 거의 비슷한 것을 볼 수 있다!!
